In [101]:
import numpy as np

import matplotlib as mpl
mpl.use('Qt5Agg')

import matplotlib.pyplot as plt
plt.ion()

# for the symbolic manipulation of jacobian
import sympy as sp
# from sympy import symbols
from sympy import sin, cos, asin, acos, pi, atan2, sqrt
from sympy.utilities.lambdify import lambdify
# from sympy import Matrix

from scipy.optimize import minimize
from scipy.optimize import fsolve

import time

import odrive
import odrive.utils
import odrive.enums

In [102]:
## Motor constants
K_T = 0.0285;                 # Nm / A
peak_amp = 30;              # A
peak_torque = K_T * peak_amp; # Nm

l1 = 0.09;                  # m 
l2 = 0.16;                  # m
w = 0.07;                   # m


In [14]:
def T(theta, x, y):
    """
    Function to return an arbitrary transformation matrix 
    This is for sympy symbolic calculation
    """
    return sp.Matrix([[sp.cos(theta), -sp.sin(theta), x], 
                      [sp.sin(theta), sp.cos(theta), y],
                      [0, 0, 1]])

def sym_to_np(T):
    return np.array(T).astype(np.float64)



## FK Through transformation matrices

In [15]:
(thetaL_sym, 
 thetaR_sym, 
 link1_sym, 
 link2_sym, 
 width_sym) = sp.symbols("""thetaL_sym 
                            thetaR_sym 
                            link1_sym 
                            link2_sym 
                            width_sym""", real = True)

In [16]:
x_r = width_sym/2 + link1_sym*sp.cos(thetaR_sym)
x_l = -width_sym/2 + link1_sym*sp.cos(thetaL_sym)

y_r = link1_sym*sp.sin(thetaR_sym)
y_l = link1_sym*sp.sin(thetaL_sym)

theta3_sym = sp.atan2(y_r - y_l, x_r - x_l)
L = sp.sqrt((x_l - x_r)**2 + (y_l - y_r)**2)

FK = T(thetaL_sym, -width_sym/2, 0)@T(-(thetaL_sym - theta3_sym), link1_sym, 0)@sp.Matrix([L/2, sp.sqrt(link2_sym**2 - (L/2)**2), 1])
FK = FK[:2,:]
FK.simplify()

#cartesian
FK = FK.subs([(link1_sym,l1), (link2_sym,l2), (width_sym,w)])
J = FK.jacobian((thetaR_sym, thetaL_sym))
J_fast = lambdify((thetaR_sym, thetaL_sym), J)
FK_fast = lambdify((thetaR_sym, thetaL_sym), FK)

In [58]:
xy = FK_fast(np.pi/2, np.pi/2)

J_new = J_fast(np.pi/2, np.pi/2)

print(xy)
print(J_new)

[[5.5109106e-18]
 [2.4612495e-01]]
[[-0.045      -0.045     ]
 [ 0.01008807 -0.01008807]]


In [ ]:
#polar

FK_polar = sp.Matrix([sp.sqrt(FK[0,0]**2 + FK[1,0]**2), sp.atan2(FK[0,0], FK[1,0])])
FK_polar_fast = lambdify((thetaR_sym, thetaL_sym), FK_polar)
J_polar = FK_polar.jacobian([thetaR_sym, thetaL_sym]).evalf()
J_pol_fast = lambdify((thetaR_sym, thetaL_sym), J_polar)

## IK through optimization

In [8]:
def IK_5_link(x, y, l1 = l1, l2 = l2, w = w):
    
    def leg_wide(var):
        return np.linalg.norm([var[0], var[1] - np.pi])
    
    def x_constraint_equation(var):
        # should be equal to zero when the 
        return l1**2 - l2**2 + (x - w/2)**2 + y**2 - 2*l1*(y*np.sin(var[0]) + (x - w/2)*np.cos(var[0]))

    def y_constraint_equation(var):
        return l1**2 - l2**2 + (x + w/2)**2 + y**2 - 2*l1*(y*np.sin(var[1]) + (x + w/2)*np.cos(var[1]))

    
    res = minimize(leg_wide, (0.1, 9*np.pi/10), method="SLSQP", constraints= ({"type": "eq", "fun": x_constraint_equation}, 
                                                                               {"type": "eq", "fun": y_constraint_equation}))
    
    return (res, np.linalg.norm([x_constraint_equation(res.x), y_constraint_equation(res.x)]))


In [9]:
def internal_angles(thetaR, thetaL, l1 = l1, l2 = l2, w = w):
    """
    Solves for the internal angles of the leg so that we can visualize
    """
    def sys(x): 
        return (w + l1*np.cos(thetaR) + l2*np.cos(x[0]) - l1*np.cos(thetaL) - l2*np.cos(x[1]),
                l1*np.sin(thetaR) + l2*np.sin(x[0]) - l1*np.sin(thetaL) - l2*np.sin(x[1]))

    alphaR, alphaL = fsolve(sys, (np.pi/2, np.pi/2))
    
    alphaR = alphaR % (2*np.pi)
    alphaL = alphaL % (2*np.pi)
        
    # Copmute FK for checking
    x = w/2 + l1*np.cos(thetaR) + l2*np.cos(alphaR);
    y = l1*np.sin(thetaR) + l2*np.sin(alphaR);

    return (alphaR, alphaL, x, y)

thetaR = .5
thetaL = np.pi

(alphaR, alphaL, x, y) = internal_angles(thetaR, thetaL)

# Should produce
# alphaL
# Out[17]: 0.8878073988680342

# alphaR
# Out[18]: 2.611036674795031


In [21]:
def draw_robot(thetaR, thetaL, link1 = l1, link2 = l2, width = w, ax = None):
    
    # Solve for internal angles
    (alphaR, alphaL, x, y) = internal_angles(thetaR, thetaL)

    def pol2cart(rho, phi):
        x = rho * np.cos(phi)
        y = rho * np.sin(phi)
        return(x, y)

    if ax is None:
        ax = plt.gca()
        ax.cla()
    

    ax.plot(-width/2, 0, 'ok')
    ax.plot(width/2, 0, 'ok')

    ax.plot([-width/2, 0], [0, 0], 'k')
    ax.plot([width/2, 0], [0, 0], 'k')
    
    ax.plot(-width/2 + np.array([0, link1*np.cos(thetaL)]), [0, link1*np.sin(thetaL)], 'k')
    ax.plot(width/2 + np.array([0, link1*np.cos(thetaR)]), [0, link1*np.sin(thetaR)], 'k')

    ax.plot(-width/2 + link1*np.cos(thetaL) + np.array([0, link2*np.cos(alphaL)]), \
             link1*np.sin(thetaL) + np.array([0, link2*np.sin(alphaL)]), 'k');
    
    ax.plot(width/2 + link1*np.cos(thetaR) + np.array([0, link2*np.cos(alphaR)]), \
             np.array(link1*np.sin(thetaR) + np.array([0, link2*np.sin(alphaR)])), 'k');
    
    ax.plot(x, y, 'ro');
# plt.set_aspect
    ax.set_aspect('equal')
#     plt.plot(x_end, y_end, 'go');

    ax.axis([-.3,.3,-.1,.3])
    
thetaR = np.pi/4
thetaL = 3*np.pi/4

draw_robot(thetaR, thetaL)

## Exploration of torques and forces 

In [114]:
##### List of starting positions to compare required torque to provide pre-determined force
# calculate theta values from position to calculate jacobian
# calculate torques and plot
xs = [-0.08, -0.05, 0.0, 0.05, 0.08]
ys = [0.08, 0.15]
tau = np.array([[20.0*K_T],[5.0*K_T]])
currents = []
forces_x = []
forces_y = []
for x in xs:
    for y in ys:
        res = IK_5_link(x, y)
        thetaR = res[0].x[0]
        thetaL = res[0].x[1]
        #draw_robot(thetaR, thetaL)
        J = J_fast(thetaR, thetaL)
        #tau = J.T @ F
        N = np.linalg.inv(J.T)
        F = N @ tau
        forces_x.append(F[0,0])
        forces_y.append(F[1,0])
        
x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
plt.bar(x, forces_y, tick_label=x, width=0.5)
plt.xlabel("Position")
plt.ylabel("Force (y direction)")
#draw_robot(-1.0218044064009442, 2.51139234448664)

Text(0, 0.5, 'Force (y direction)')